# Exploring the Prosper Loan dataset
## Objective and Overview of dataset
In this project I am exploring a dataset containing loan details that have been borrowed on the p2p lending platform 'Prosper'. The dataset contains details of about 113,000 loans such as credit scores, grades and ratings, borrower rates, APR (Annual Percentage Rate), details of last occupation status and many more. My objective with this analysis is to better understand financial services datasets and the common vocabulary used. Also to gain some intuition for data that is captured as part of the loan application and approval process. The process I am following here would be to use visualizations to gain insight.

There are 81 attributes and 113k observations in the dataset.


- Top 10 borrower states
- Loan status categories
- credit grade distribution
- Borrower rate vs Loan status
- Borrower state vs Loan status
- Employment status vs Loan status
- Loan status vs Credit Grade (heatmap)

## Top borrower states


- Top 10 borrower states
- Loan status categories
- credit grade distribution
- Borrower rate vs Loan status
- Borrower state vs Loan status
- Employment status vs Loan status
- Loan status vs Credit Grade (heatmap)